In [1]:
from tg.grammar_ru import *
import os
import pandas as pd
import numpy as np
from yo_fluq_ds import *
import subprocess
import ipywidgets as widgets

rucoco_dir = Loc.data_cache_path/'rucoco'
db = DataBundle.load(rucoco_dir/'bundle-full.zip')
word_to_normal_form = db.words.set_index('word').nf_n.to_dict()
LEN = len(set(word_to_normal_form.values()))

In [2]:
len(word_to_normal_form), LEN

(82560, 25841)

In [3]:
DATASET = rucoco_dir/'ds'

In [4]:
from jordan_lib.preprocessing import LibrusecTranslator

lenta_path = rucoco_dir/'lenta-ru-news.csv.gz'
ltr = LibrusecTranslator(lenta_path, word_to_normal_form)

In [5]:
from glove_helpers.prep import CoocSelector, MatrixCollector, VocabStatsCollector

if True:
    cs = CoocSelector(15, LEN, True)
    cl = MatrixCollector(LEN, DATASET/'matrix', 1000)
    vsc = VocabStatsCollector(LEN, DATASET/'vocab', 1000)
    (ltr
     .get_dfs()
     .feed(fluq.with_progress_bar())
     .take(50000)
     .foreach_and_continue(vsc.observe)
     .select(cs.compute_cooc)
     .foreach(cl.observe)
    )

  0%|          | 0/301871 [00:00<?, ?it/s]

In [10]:
import numpy as np
np.save(cl.file, cl.matrix)

In [11]:
import numpy as np
from glove_helpers.prep import MatrixCollector
import time

start = time.time()
if True:
    os.makedirs(DATASET/'assembled', exist_ok=True)
    (MatrixCollector
     .matrix_to_glove_tuples(cl.matrix, LEN, True)
     .feed(fluq.with_progress_bar())
     .where(lambda z: z[1]<z[2])
     .select(lambda z: (z[0], z[3]))
     .feed(fluq.partition_by_count(500000))
     .select(lambda z: pd.DataFrame(z, columns=['idx','value']).set_index('idx'))
     .with_indices()
     .foreach(lambda z: z.item.to_parquet(DATASET/'assembled'/str(z.index)))
    )

  0%|          | 0/12998188 [00:00<?, ?it/s]